In [207]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [208]:
trn = pd.read_csv('./data/train_ver2.csv')
tst = pd.read_csv('./data/test_ver2.csv')

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [209]:
prods = trn.columns[24:].tolist()

In [210]:
trn[prods] = trn[prods].fillna(0.0).astype(np.int8)

In [211]:
no_product = trn[prods].sum(axis=1) == 0
trn = trn[~no_product]

In [212]:
for col in trn.columns[24:]:
    tst[col] = 0
df = pd.concat([trn, tst], axis=0)

In [213]:
features = []

In [214]:
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']
for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel=-99)
features += categorical_cols

In [215]:
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace=True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

In [216]:
features += ['age','antiguedad','renta','ind_nuevo','indrel','indrel_1mes','ind_actividad_cliente']

In [217]:
df['fecha_alta_month'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year'] = df['fecha_alta'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['fecha_alta_month', 'fecha_alta_year']

df['ult_fec_cli_1t_month'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[1])).astype(np.int8)
df['ult_fec_cli_1t_year'] = df['ult_fec_cli_1t'].map(lambda x: 0.0 if x.__class__ is float else float(x.split('-')[0])).astype(np.int16)
features += ['ult_fec_cli_1t_month', 'ult_fec_cli_1t_year']

In [218]:
df.fillna(-99, inplace=True)

In [219]:
def date_to_int(str_date):
    Y, M, D = [int(a) for a in str_date.strip().split("-")] 
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date

In [220]:
df['int_date'] = df['fecha_dato'].map(date_to_int).astype(np.int8)

In [221]:
df_lag = df.copy()
df_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date'] else col for col in df.columns ]
df_lag['int_date'] += 1

In [222]:
df_trn = df.merge(df_lag, on=['ncodpers','int_date'], how='left')

In [223]:
del df, df_lag

In [224]:
for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace=True)
df_trn.fillna(-99, inplace=True)

In [225]:
features += [feature + '_prev' for feature in features]
features += [prod + '_prev' for prod in prods]

In [226]:
use_dates = ['2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28', '2016-05-28']
trn = df_trn[df_trn['fecha_dato'].isin(use_dates)]
tst = df_trn[df_trn['fecha_dato'] == '2016-06-28']
del df_trn

In [227]:
X = []
Y = []
for i, prod in enumerate(prods):
    prev = prod + '_prev'
    prX = trn[(trn[prod] == 1) & (trn[prev] == 0)]
    prY = np.zeros(prX.shape[0], dtype=np.int8) + i
    X.append(prX)
    Y.append(prY)
XY = pd.concat(X)
Y = np.hstack(Y)
XY['y'] = Y

In [228]:
vld_date = '2016-05-28'
XY_trn = XY[XY['fecha_dato'] != vld_date]
XY_vld = XY[XY['fecha_dato'] == vld_date]

In [229]:
param = {
    'booster': 'gbtree',
    'max_depth': 8,
    'nthread': 4,
    'num_class': len(prods),
    'objective': 'multi:softprob',
    'silent': 1,
    'eval_metric': 'mlogloss',
    'eta': 0.1,
    'min_child_weight': 10,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.9,
    'seed': 2018,
    }

In [231]:
X_trn = XY_trn[features].values
Y_trn = XY_trn['y'].values
dtrn = xgb.DMatrix(X_trn, label=Y_trn, feature_names=features)

X_vld = XY_vld[features].values
Y_vld = XY_vld['y'].values
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)

In [232]:
watch_list = [(dtrn, 'train'), (dvld, 'eval')]
model = xgb.train(param, dtrn, num_boost_round=1000, evals=watch_list, early_stopping_rounds=20)

[11:00:04] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-mlogloss:2.68092	eval-mlogloss:2.68980
[1]	train-mlogloss:2.46791	eval-mlogloss:2.48087
[2]	train-mlogloss:2.29254	eval-mlogloss:2.30787
[3]	train-mlogloss:2.15647	eval-mlogloss:2.17447
[4]	train-mlogloss:2.04496	eval-mlogloss:2.06416
[5]	train-mlogloss:1.94649	eval-mlogloss:1.96528
[6]	train-mlogloss:1.86704	eval-mlogloss:1.88623
[7]	train-mlogloss:1.79857	eval-mlogloss:1.81833
[8]	train-mlogloss:1.73422	eval-mlogloss:1.75382
[9]	train-mlogloss:1.67965	eval-mlogloss:1.69974
[10]	train-mlogloss:1.62933	eval-mlogloss:1.64932
[11]	train-mlogloss:1.58643	eval-mlogloss:1.60678
[12]	train-mlogloss:1.54636	eval-mlogloss:1.56683
[13]

[156]	train-mlogloss:1.00300	eval-mlogloss:1.08808
[157]	train-mlogloss:1.00243	eval-mlogloss:1.08807
[158]	train-mlogloss:1.00188	eval-mlogloss:1.08799
[159]	train-mlogloss:1.00129	eval-mlogloss:1.08798
[160]	train-mlogloss:1.00069	eval-mlogloss:1.08800
[161]	train-mlogloss:1.00002	eval-mlogloss:1.08800
[162]	train-mlogloss:0.99955	eval-mlogloss:1.08793
[163]	train-mlogloss:0.99911	eval-mlogloss:1.08795
[164]	train-mlogloss:0.99852	eval-mlogloss:1.08793
[165]	train-mlogloss:0.99795	eval-mlogloss:1.08789
[166]	train-mlogloss:0.99730	eval-mlogloss:1.08791
[167]	train-mlogloss:0.99680	eval-mlogloss:1.08786
[168]	train-mlogloss:0.99624	eval-mlogloss:1.08780
[169]	train-mlogloss:0.99573	eval-mlogloss:1.08776
[170]	train-mlogloss:0.99522	eval-mlogloss:1.08773
[171]	train-mlogloss:0.99476	eval-mlogloss:1.08767
[172]	train-mlogloss:0.99415	eval-mlogloss:1.08761
[173]	train-mlogloss:0.99358	eval-mlogloss:1.08763
[174]	train-mlogloss:0.99309	eval-mlogloss:1.08758
[175]	train-mlogloss:0.99254	ev

In [233]:
import pickle
pickle.dump(model, open("./model/xgb.baseline.pkl", "wb"))
best_ntree_limit = model.best_ntree_limit

In [235]:
vld = trn[trn['fecha_dato'] == vld_date]
ncodpers_vld = vld['ncodpers'].values

In [237]:
for prod in prods:
    prev = prod + '_prev'
    padd = prod + '_add'
    vld[padd] = vld[prod] - vld[prev]    
add_vld = vld[[prod + '_add' for prod in prods]].values
add_vld_list = [list() for i in range(len(ncodpers_vld))]

<ipython-input-237-fda838761a90>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vld[padd] = vld[prod] - vld[prev]


In [238]:
count_vld = 0
for ncodper in range(len(ncodpers_vld)):
    for prod in range(len(prods)):
        if add_vld[ncodper, prod] > 0:
            add_vld_list[ncodper].append(prod)
            count_vld += 1

In [239]:
def apk(actual, predicted, k=7, default=0.0):
    # AP@7なので、最大7個まで使用します。
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        # 点数を付与する条件は次のとおり :
        # 予測値が正答に存在し (‘p in actual’)
        # 予測値に重複がなければ (‘p not in predicted[:i]’) 
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    # 正答値が空白である場合、ともかく 0.0点を返します。
    if not actual:
        return default

    # 正答の個数(len(actual))として average precisionを求めます。
    return score / min(len(actual), k)

def mapk(actual, predicted, k=7, default=0.0):
    # list of listである正答値(actual)と予測値(predicted)から顧客別 Average Precisionを求め, np.mean()を通して平均を計算します。
    return np.mean([apk(a, p, k, default) for a, p in zip(actual, predicted)]) 

In [240]:
print(mapk(add_vld_list, add_vld_list, 7, 0.0))

0.04266379915553903


In [247]:
X_all = XY[features].values
Y_all = XY['y'].values
dall = xgb.DMatrix(X_all, label=Y_all, feature_names=features)
watch_list = [(dall, 'train')]

In [248]:
best_ntree_limit = int(best_ntree_limit * (len(XY_trn) + len(XY_vld)) / len(XY_trn))

In [249]:
model = xgb.train(param, dall, num_boost_round=best_ntree_limit, evals=watch_list)

[11:23:34] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-mlogloss:2.68170
[1]	train-mlogloss:2.46856
[2]	train-mlogloss:2.29314
[3]	train-mlogloss:2.15684
[4]	train-mlogloss:2.04526
[5]	train-mlogloss:1.94643
[6]	train-mlogloss:1.86679
[7]	train-mlogloss:1.79815
[8]	train-mlogloss:1.73355
[9]	train-mlogloss:1.67876
[10]	train-mlogloss:1.62828
[11]	train-mlogloss:1.58542
[12]	train-mlogloss:1.54514
[13]	train-mlogloss:1.51041
[14]	train-mlogloss:1.47838
[15]	train-mlogloss:1.45046
[16]	train-mlogloss:1.42268
[17]	train-mlogloss:1.39846
[18]	train-mlogloss:1.37757
[19]	train-mlogloss:1.35661
[20]	train-mlogloss:1.33794
[21]	train-mlogloss:1.32075
[22]	train-mlogloss:1.30379
[23]	train

In [250]:
print("Feature importance:")
for kv in sorted([(k,v) for k,v in model.get_fscore().items()], key=lambda kv: kv[1], reverse=True):
    print(kv)

Feature importance:
('renta', 19248)
('age', 17869)
('antiguedad', 17098)
('age_prev', 12013)
('antiguedad_prev', 11969)
('fecha_alta_month', 11372)
('nomprov', 10939)
('fecha_alta_year', 8710)
('renta_prev', 7744)
('canal_entrada', 7244)
('nomprov_prev', 5864)
('canal_entrada_prev', 4331)
('fecha_alta_month_prev', 3894)
('ind_recibo_ult1_prev', 3285)
('ind_ecue_fin_ult1_prev', 3062)
('sexo', 3046)
('fecha_alta_year_prev', 2971)
('ind_cco_fin_ult1_prev', 2968)
('ind_cno_fin_ult1_prev', 2724)
('ind_tjcr_fin_ult1_prev', 2089)
('segmento', 2084)
('ind_reca_fin_ult1_prev', 1997)
('segmento_prev', 1825)
('ind_nom_pens_ult1_prev', 1650)
('tiprel_1mes', 1562)
('ind_valo_fin_ult1_prev', 1540)
('ind_nomina_ult1_prev', 1437)
('ind_ctop_fin_ult1_prev', 1433)
('ind_dela_fin_ult1_prev', 1405)
('ind_actividad_cliente', 1293)
('sexo_prev', 1221)
('tiprel_1mes_prev', 1102)
('ind_ctpp_fin_ult1_prev', 1072)
('ind_fond_fin_ult1_prev', 1001)
('ind_ctma_fin_ult1_prev', 885)
('ind_actividad_cliente_prev', 8

In [252]:
X_tst = tst[features].values
dtst = xgb.DMatrix(X_tst, feature_names=features)
preds_tst = model.predict(dtst, ntree_limit=best_ntree_limit)
ncodpers_tst = tst['ncodpers'].values
preds_tst = preds_tst - tst[[prod + '_prev' for prod in prods]].values

In [253]:
submit_file = open('./model/xgb.baseline.2015-06-28', 'w')
submit_file.write('ncodpers,added_products\n')
for ncodper, pred in zip(ncodpers_tst, preds_tst):
    y_prods = [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods = sorted(y_prods, key=lambda a: a[0], reverse=True)[:7]
    y_prods = [p for y,p,ip in y_prods]
    submit_file.write('{},{}\n'.format(int(ncodper), ' '.join(y_prods)))